# developing and testing pynetstim

In [ ]:
from brainsight import BrainsightSessionFile
import nibabel as nib
import numpy as np
from utils import Coords
from surfer import Brain
from mayavi import mlab
import matplotlib.pyplot as plt

## Testing BrainsightSessionFile and rapid visualization of stimulations on brain
Date: 2018-09-21 12:10:16 

In [5]:
subject = 'Broad_69_LD'
subjects_dir = '/Users/stadayon/Dropbox/projects/broad/data/freesurfer/subjects'

lh_cortex_vertices, lh_cortex_faces = nib.freesurfer.read_geometry('{subjects_dir}/{subject}/surf/lh.pial'.format(
    subjects_dir=subjects_dir, subject=subject))

rh_cortex_vertices, rh_cortex_faces = nib.freesurfer.read_geometry('{subjects_dir}/{subject}/surf/rh.pial'.format(
    subjects_dir=subjects_dir, subject=subject))

# generate skin surface using mri_watershed ( don't use -useSRAS as it takes a long time and I don't know why).
# the generated skin is in RAS coordinate. We can use pynetstim Coords class to read the skin vertices and convert it
# to tkr-ras_coords.
skin_vertices, skin_faces = nib.freesurfer.read_geometry(
    '/Users/stadayon/Dropbox/projects/broad/data/anat/Broad_69_LD/lh.brainsurf_outer_skin_surface')
skin_vertices = Coords(skin_vertices, subject,
                       subjects_dir, False).coords['tkr-ras_coords']

# reading brainsight session file using pynetstim
bs = BrainsightSessionFile('/Users/stadayon/Dropbox/projects/broad/data/sessions_data/69_LD Exported Brainsight Data.txt',
                           './temp')
df = bs.get_table('samples')
df_sess1 = df[df.session_name == 'Session 1']

# converting stimulation sites to surface coordinate using pynetstim Coords class.
stim_coords = Coords(df[['loc_x', 'loc_y', 'loc_z']],
                     subject, subjects_dir).coords['tkr-ras_coords']


# set offset off otherwise, pysurfer tries to move the hemispheres so you can visualize the medial sid better
brain = Brain(subject, subjects_dir=subjects_dir,
              hemi='both', surf='pial', offset=False)

# adding annotations
brain.add_annotation('Yeo2011_7Networks_N1000', hemi='lh', borders=False)
brain.add_annotation('Yeo2011_7Networks_N1000', hemi='rh',
                     borders=False, remove_existing=False)

# as we are  not trying to reinvent the wheel, It is better to stick to pysurfer for visualizaiton. However
# this is a sanity check whether things works as we expect them to work.

# mlab.triangular_mesh(lh_cortex_vertices[:,0],lh_cortex_vertices[:,1],lh_cortex_vertices[:,2],lh_cortex_faces,reset_zoom=False)
# mlab.triangular_mesh(rh_cortex_vertices[:,0],rh_cortex_vertices[:,1],rh_cortex_vertices[:,2],rh_cortex_faces,reset_zoom=False)

# visualization of stimulation sites using points3d
mlab.triangular_mesh(skin_vertices[:, 0], skin_vertices[:, 1], skin_vertices[:, 2],
                     skin_faces, opacity=.4, color=(1, 1, .4), reset_zoom=False)
mlab.points3d(stim_coords[:, 0], stim_coords[:, 1],
              stim_coords[:, 2], scale_factor=4, reset_zoom=False)
mlab.show()